In [1]:
from analysis.data_gen_utils import download_IBL, extract_IBL, make_dataset, combine_datasets

2023-10-09 18:39:05.731094: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-09 18:39:05.731127: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-09 18:39:05.731149: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-10-09 18:39:05.737613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 18:39:06.598260: W tensorflow/compiler/

In [2]:
# download IBL data to a location and destripe/process using the session's pid
pid_sess1 = 'dab512bd-a02d-4c1f-8dbc-9155a163efc0'
pid_sess2 = 'febb430e-2d50-4f83-87a0-b5ffbb9a4943'
t_window = [0, 21]
save_folder_sess1 = '/media/cat/data/IBL_data_CEED/dab512bd-a02d-4c1f-8dbc-9155a163efc0'
save_folder_sess2 = '/media/cat/data/IBL_data_CEED/febb430e-2d50-4f83-87a0-b5ffbb9a4943'
bin_file_sess1, meta_file_sess1 = download_IBL(pid=pid_sess1, t_window = t_window, save_folder=save_folder_sess1)
bin_file_sess2, meta_file_sess2 = download_IBL(pid=pid_sess2, t_window = t_window, save_folder=save_folder_sess2)

(385, 630000)
2023-10-09 18:39:19.525 WARNING  [spikeglx.py:104] /home/cole/Downloads/ONE/alyx.internationalbrainlab.org/cache/ap/danlab/Subjects/DY_016/2020-09-12/001/raw_ephys_data/probe00/chunk_000000_to_000020/_spikeglx_ephysData_g0_t0.imec0.ap.stream.cbin : meta data and compressed chunks dont checkout
File duration: expected 3668.9780333333333, actual 21.0
Will attempt to fudge the meta-data information.
File duration: expected 3668.9780333333333, actual 21.0
Will attempt to fudge the meta-data information.


ValueError: negative dimensions are not allowed

In [ ]:
# extract the necessary spike index and other items to make a dataset
spike_idx_sess1, geom_sess1, chan_idx_sess1, templates_sess1 = extract_IBL(bin_fp=bin_file_sess1, meta_fp=meta_file_sess1, pid=pid_sess1, t_window=t_window)
spike_idx_sess2, geom_sess2, chan_idx_sess2, templates_sess2 = extract_IBL(bin_fp=bin_file_sess2, meta_fp=meta_file_sess2, pid=pid_sess2, t_window=t_window)

In [ ]:
# session 1 units to get data from and dataset save path
selected_units_sess1 = [10, 20, 30]
dataset_folder_sess1 = '/ds1'

# make a dataset for training
# will create a folder with the spike, probe channel number, and corresponding channel location datasets in the train, val, test splits
# optionally also saves out spatial and temporal noise covariance matrices
make_dataset(bin_path=bin_file_sess1, spike_index=spike_idx_sess1, geom=geom_sess1, save_path=dataset_folder_sess1, chan_index=chan_idx_sess1, 
             templates=templates_sess1, unit_ids=selected_units_sess1, train_num=200, val_num=0, test_num=200, save_covs=True,
             num_chans_extract=21, plot=False, normalize=False, shift=False, do_split=True,
             save_var_num=False)

In [ ]:
selected_units_sess2 = [100, 110, 120]
dataset_folder_sess2 = '/ds2'

# make dataset 2 for training
make_dataset(bin_path=bin_file_sess2, spike_index=spike_idx_sess2, geom=geom_sess2, save_path=dataset_folder_sess2, chan_index=chan_idx_sess2, 
             templates=templates_sess2, unit_ids=selected_units_sess2, train_num=200, val_num=0, test_num=200, save_covs=True,
             num_chans_extract=21, plot=False, normalize=False, shift=False, do_split=True, 
             save_var_num=False)

In [ ]:
combined_ds_path = '/combined'

# combine the two training datasets into a larger one for more unit diversity
combine_datasets(dataset_folder_sess1, dataset_folder_sess2, combined_ds_path)

In [ ]:
inference_ds_path = '/inference'
selected_units_inf = [200, 210, 220]

# make a dataset for spike sorting inference
# will save out only spikes, channel numbers, and channel locations for a spikes test set (only pass in test_num)
# save_var_num flag will allow units with < test_num spikes in the recording to be saved out in the test set as well (change to min and max spikes)
# for cell type datasets the normalize Flag can be set to True
make_dataset(bin_path=bin_file_sess1, spike_index=spike_idx_sess1, geom=geom_sess1, save_path=inference_ds_path, chan_index=chan_idx_sess1, 
             templates=templates_sess1, unit_ids=selected_units_inf, test_num=200, save_covs=True,
             num_chans_extract=21, do_data_split=False, normalize=False, save_var_num=False)